## 1. Installation Packages

In [1]:
# Install Gensim

!pip install gensim


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 req

ERROR: Could not find a version that satisfies the requirement python-levensthein (from versions: none)
ERROR: No matching distribution found for python-levensthein


In [2]:
# Install python-Levenshtein

!pip install python-Levenshtein


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.5 MB/s eta 0:00:00


## 2. Upload Dataset

In [3]:
# Upload the dataset

from google.colab import files
uploaded = files.upload()


Saving reviews_Cell_Phones_and_Accessories_5.json.gz to reviews_Cell_Phones_and_Accessories_5.json.gz


In [4]:
# Unzip

import gzip
import json

file_name = 'reviews_Cell_Phones_and_Accessories_5.json.gz'

with gzip.open(file_name, 'rt', encoding='utf-8') as f:
    # Assuming each line in the file is a separate JSON object
    data = [json.loads(line) for line in f]

# Now 'data' is a list of dictionaries, each representing a JSON object
print(f"Successfully loaded {len(data)} JSON objects.")
# You can now work with the 'data' variable


Successfully loaded 194439 JSON objects.


In [6]:
# For example, to print the first object:

data[0]

{'reviewerID': 'A30TL5EWN6DFXT',
 'asin': '120401325X',
 'reviewerName': 'christina',
 'helpful': [0, 0],
 'reviewText': "They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again",
 'overall': 4.0,
 'summary': 'Looks Good',
 'unixReviewTime': 1400630400,
 'reviewTime': '05 21, 2014'}

In [12]:

import pandas as pd
import gensim
from gensim.models import Word2Vec


# Define the Dataset

df = pd.read_json(file_name, lines=True)
print(df.head())
print("\nShape :",df.shape)


       reviewerID        asin      reviewerName helpful  \
0  A30TL5EWN6DFXT  120401325X         christina  [0, 0]   
1   ASY55RVNIL0UD  120401325X          emily l.  [0, 0]   
2  A2TMXE2AFO7ONB  120401325X             Erica  [0, 0]   
3   AWJ0WZQYMYFQ4  120401325X                JM  [4, 4]   
4   ATX7CZYFXI1KW  120401325X  patrice m rogoza  [2, 3]   

                                          reviewText  overall  \
0  They look good and stick good! I just don't li...        4   
1  These stickers work like the review says they ...        5   
2  These are awesome and make my phone look so st...        5   
3  Item arrived in great time and was in perfect ...        4   
4  awesome! stays on, and looks great. can be use...        5   

                                     summary  unixReviewTime   reviewTime  
0                                 Looks Good      1400630400  05 21, 2014  
1                      Really great product.      1389657600  01 14, 2014  
2                         

## 3. Data Preprocessing

In [11]:
# Check is there any Null

Null = df.isnull().sum()
print(Null)

print("\n")


# Info of the Dataset

print("\n",df.info())

print("\n")


# Decribe the Dataset

print(df.describe())


reviewerID           0
asin                 0
reviewerName      3519
helpful              0
reviewText           0
overall              0
summary              0
unixReviewTime       0
reviewTime           0
dtype: int64


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194439 entries, 0 to 194438
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   reviewerID      194439 non-null  object
 1   asin            194439 non-null  object
 2   reviewerName    190920 non-null  object
 3   helpful         194439 non-null  object
 4   reviewText      194439 non-null  object
 5   overall         194439 non-null  int64 
 6   summary         194439 non-null  object
 7   unixReviewTime  194439 non-null  int64 
 8   reviewTime      194439 non-null  object
dtypes: int64(2), object(7)
memory usage: 13.4+ MB

 None


             overall  unixReviewTime
count  194439.000000    1.944390e+05
mean        4.129912    1.368714e+09
st

In [13]:
# Preprocess using Gensim

df.reviewText = df.reviewText.apply(gensim.utils.simple_preprocess)

df.reviewText.head()


,reviewText
0,"[they, look, good, and, stick, good, just, don..."
1,"[these, stickers, work, like, the, review, say..."
2,"[these, are, awesome, and, make, my, phone, lo..."
3,"[item, arrived, in, great, time, and, was, in,..."
4,"[awesome, stays, on, and, looks, great, can, b..."


## 4. Train the Model

In [14]:
# Initialize the model

Model = Word2Vec(

    window=10,
    min_count=2,
    workers=4,

)


# Build a Vocabulary

Vocabulary = Model.build_vocab(df.reviewText, progress_per=1000)


In [15]:
# Corpus Count

Count = Model.corpus_count
print("Corpus Count :", Count)


# Epochs

Epochs = Model.epochs
print("\nEpochs :", Epochs)


Corpus Count : 194439

Epochs : 5


In [16]:
# Train the Model

Model.train(df.reviewText, total_examples=Count, epochs=Epochs)


(61505362, 83868975)

In [19]:
# To Check

Similarity = Model.wv.similarity(w1="good", w2="great")
print("Similarity :", Similarity)


Similar = Model.wv.most_similar("bad")
print("\nSimilar :", Similar)


Similarity : 0.77861094

Similar : [('terrible', 0.6834560036659241), ('shabby', 0.6276258826255798), ('horrible', 0.5992334485054016), ('good', 0.5817492604255676), ('awful', 0.5634989738464355), ('crappy', 0.5514112710952759), ('disappointing', 0.5277916193008423), ('cheap', 0.525065004825592), ('funny', 0.5231936573982239), ('okay', 0.5168481469154358)]
